## Step 1 : Web Scraping 

In [2]:
#Import dependencies 
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
from random import randint
from time import sleep 

In [3]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\eenab\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [4]:
#Create empty dictionary to hold all variables 
mars_data = {}

# title : string
# paragraph : string
# image :string
# pd.to_html -- run on the table they provide : string
# hemispheres: list of dictionaries


## NASA Mars News

In [5]:
# Visit Nasa news url through splinter module
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
sleep(randint(1,10))

In [6]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(browser.html, 'html.parser')

In [7]:
#Find latest news title 
news_title = soup.find_all('div', class_='content_title')
news_title[1].get_text()
title = news_title[1].get_text()
mars_data["title"] = title
title

'Testing Proves Its Worth With Successful Mars Parachute Deployment'

In [8]:
#Find latest news description
news_article = soup.find('div', class_='article_teaser_body')
news_p= news_article.get_text()
mars_data["news_p"] = news_p
news_p

'The giant canopy that helped land Perseverance on Mars was tested here on Earth at NASA’s Wallops Flight Facility in Virginia.'

In [9]:
# browser.quit()

## Featured Image

In [10]:
# Visit JPL Mars Space Images
jpl_img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_img_url)
sleep(randint(1,10))


In [11]:
# Sort for latest Mars photos in gallery 
browser.find_by_id('filter_Mars').first.click()
sleep(randint(1,10))

In [12]:
#Set up new html/bs
featured_img_html = browser.html
img_soup = BeautifulSoup(featured_img_html, 'html')
featured_img_url = img_soup.find('img', class_="BaseImage object-contain")["src"]
print(featured_img_url)
mars_data["image"] = featured_img_url

https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24415.2e16d0ba.fill-400x400-c50.jpg


## Mars Facts

In [13]:
# Visit Mars facts url through splinter module
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)
sleep(randint(1,10))

In [22]:
# Scrape the table containing facts about the planet including Diameter, Mass, etc.and turn into HTML table string
facts_table = pd.read_html(facts_url)

mars_facts= facts_table[0]
# Rename columns 
mars_facts.columns = ['Description', 'Value']

mars_facts_html = mars_facts.to_html(index = False)

#insert into dictionary
mars_data["facts_table"]= mars_facts_html 


,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


## Mars Hemispheres

In [ ]:
#Set an empty list to hold urls
hemis_img_urls = []
    # {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    # {"title": "Cerberus Hemisphere", "img_url": "..."},
    # {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    # {"title": "Syrtis Major Hemisphere", "img_url": "..."},


In [ ]:
#Visit main site
hemis_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemis_url)
sleep(randint(1,10))

In [ ]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
#Locate each of the divs with hemisphere info
base_url = "https://astrogeology.usgs.gov"

results = soup.find_all("div", class_='description')
results

In [ ]:
#Below finds one link, used as an example
results[0].find('a')['href']

In [ ]:
#Runs through each hemisphere div, clicks in to find link to full size image URL, title of Hemisphere, and appends the dictionary
for result in results:
    
    hemi_title = result.find('h3').text

    img_page_url = base_url + result.find('a')['href']
    browser.visit(img_page_url)
    
    html = browser.html
    img_page_soup = BeautifulSoup(html, 'html.parser')
    
    hemi_img_url = img_page_soup.find('ul').li.a['href']
    
    hemi_img_dict = {'title': hemi_title, 
                'img_url': hemi_img_url}
    
    hemis_img_urls.append(hemi_img_dict)


In [ ]:
hemis_img_urls

In [ ]:
mars_data

In [ ]:
browser.quit()